In [ ]:
import pandas as pd
#provaxxers
#provaxxersTweets = pd.read_csv('./datasets/provaxxers/provaxxersTweets.csv', low_memory=False)
#vacinaja = pd.read_csv('./datasets/provaxxers/vacinaja.csv', low_memory=False)

#antivaxxers
#antivaxxersTweets = pd.read_csv('./datasets/antivaxxers/antivaxxersTweets.csv', low_memory=False)
#antisinovaxxers
#antisinovaxxersTweets = pd.read_csv('./datasets/antisinovaxxers/old/antisinovaxxersTweets.csv', low_memory=False)


import nltk
from nltk import tokenize
import numpy as np 
from string import punctuation
import unidecode
stemmer = nltk.RSLPStemmer()

# Removendo hashtags, menções a usuários, numeros, termos curtos e links

def proccess_text(raw_text):
    
    twitterData = pd.DataFrame(raw_text) 
    twitterData['processed_text'] = twitterData.text.str.lower() \
                                                    .str.replace(r'(http\S+)', '') \
                                                    .str.replace(r'@[\w]*', '') \
                                                    .str.replace(r'#[\w]*','') 

    textWords = ' '.join([text for text in twitterData.processed_text])

    # Removendo acentuação
    textWords = [unidecode.unidecode(text) for text in twitterData.processed_text ]

    # Criando lista com palavras e caracteres (stopwords) a serem removidos do texto
    stopWords = nltk.corpus.stopwords.words("portuguese")

    # Separando a pontuação das palavras
    punctSeparator = tokenize.WordPunctTokenizer()
    punctuationList = list()
    for punct in punctuation:
        punctuationList.append(punct)

    personalList=['predictions']    

    stopWords =  stopWords + punctuationList + personalList
    #stopWords =   punctuationList + personalList


    # Iterando o texto removendo as stopwords
    trasnformedText = list()
    
    for text in textWords:
        newText = list()   
        #text = text.lower()
        textWords = punctSeparator.tokenize(text)
        for words in textWords:
             if words not in stopWords:
                #newText.append(stemmer.stem(words))
                newText.append(words)
        trasnformedText.append(' '.join(newText))
    twitterData.processed_text = trasnformedText

   
    twitterData.processed_text = twitterData.processed_text.str.replace(r"[^a-zA-Z#]", " ") \
                                                           .replace(r"k\k", " ") #\
                                                           #.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) 
    
    return twitterData.processed_text


tweets = {'created_at': antisinovaxxersTweets.created_at, 'text': antisinovaxxersTweets.text}
rawTweets = pd.DataFrame(tweets)

rawTweets['processed_text'] = proccess_text(rawTweets.text)

processedTweets = rawTweets.drop(columns=["text"])

trasnformedText = list()
for phrase in processedTweets.processed_text:
    newPhrase = list()   
    newPhrase.append(' '.join(phrase.split()))
    for words in newPhrase:
        trasnformedText.append(''.join(newPhrase))
processedTweets.processed_text = trasnformedText

index=[x for x in processedTweets.index if processedTweets.processed_text[x].count(' ') < 4]
processedTweets = processedTweets.drop(index)

removeEmpty  = processedTweets.processed_text != ' '
processedTweets = processedTweets[removeEmpty]

processedTweets.reset_index(inplace=True)

tweets = {'created_at': processedTweets.created_at, 'text': processedTweets.processed_text}
docs = pd.DataFrame(tweets)

docs = docs.sort_values(['created_at']).reset_index()

docs = docs.drop(columns=["index"])

docs

In [ ]:
import pandas as pd
provaxxersTweets = pd.read_csv('./datasets/provaxxers/provaxxersTweets.csv', low_memory=False)
oldProvaxxersTweets = pd.read_csv('./datasets/provaxxers/old/provaxxersTweets.csv', low_memory=False)


In [ ]:
start_date ='2020-02-29T00:00:00.000Z'
end_date = '2021-05-04T00:00:00.000Z'

mask = (provaxxersTweets['created_at'] > start_date) & (provaxxersTweets['created_at'] <= end_date)
filteredTweets = provaxxersTweets.loc[mask]

In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():        
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")    
    print('There are %d GPU(s) available.' % torch.cuda.device_count())    
    print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## Computing Sentence Embeddings with BERT

In [ ]:
from sentence_transformers import SentenceTransformer, util

def computing_embeddings(docs):
    bert_model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens", device="cuda")
    bert_embeddings = bert_model.encode(docs, show_progress_bar=True,convert_to_tensor=False)
    return bert_model, bert_embeddings


In [ ]:
bert_model, bert_embeddings = computing_embeddings(docs.text)

## Dimensionality reduction with UMAP

In [ ]:
from umap import UMAP
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


def reduce_dimensionality(embeddings,n,c):
    umap_model = UMAP(n_neighbors=n, n_components=c, metric='cosine')
    umap_embeddings = umap_model.fit_transform(embeddings)
    return umap_model, umap_embeddings

In [ ]:
umap_model, umap_embeddings = reduce_dimensionality(bert_embeddings,15,5)

## Clustering with K-means

In [ ]:
from sklearn.cluster import KMeans

#kmeans = KMeans(n_clusters=40, random_state=0).fit(umap_embeddings)

## Clustering with AgglomerativeClustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
ac_model = AgglomerativeClustering(n_clusters=None,
                                   affinity='cosine', 
                                   memory=None, 
                                   connectivity=None, 
                                   compute_full_tree='auto', 
                                   linkage='average', 
                                   distance_threshold=0.04, 
                                   compute_distances=True).fit(umap_embeddings)

In [ ]:
ac_cluster = ac_model.fit_predict(umap_embeddings)

## Clustering with Fcluster

In [ ]:
#from scipy.cluster.hierarchy import ward, fcluster
#from scipy.spatial.distance import pdist

#Z = ward(pdist(umap_embeddings))
#clusters_fc = fcluster(Z, t=0.9, criterion='distance')

## Clustering with HDBSCAN

In [ ]:
import hdbscan
def custering_hdbscan(embeddings,min_cluster_size):
    hdbsan_cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, 
                              metric='euclidean',                      
                              cluster_selection_method='eom', prediction_data=True).fit_predict(embeddings)
    
    return hdbsan_cluster

In [ ]:
hdbsan_cluster = custering_hdbscan(umap_embeddings,24)

In [ ]:
c = pd.Series(hdbsan_cluster)
c = c.unique()
len(c)

# Ploting clusters

In [ ]:
plot_embedding_umap = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(bert_embeddings)

In [ ]:
import matplotlib.pyplot as plt

def ploting_clusters(embeddings,labels):
    result = pd.DataFrame(embeddings, columns=['x', 'y'])
    result['labels'] = labels
    # Visualize clusters
    fig, ax = plt.subplots(figsize=(20, 10))
    outliers = result.loc[result.labels == -1, :]
    clustered = result.loc[result.labels != -1, :]
    plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.15)
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
    plt.colorbar()

### K-means

In [ ]:
#ploting_clusters(plot_embedding_umap,kmeans.labels_)

### AgglomerativeClustering

In [ ]:
ploting_clusters(plot_embedding_umap,ac_cluster)

###  HDBSCAN

In [ ]:
ploting_clusters(plot_embedding_umap,hdbsan_cluster)

# c-TF-IDF

In [ ]:
def clustering_documents(docs,cluster_labels):
    docs_df = pd.DataFrame(docs, columns=["text"])
    docs_df['cluster'] = cluster_labels
    docs_df['doc_id'] = range(len(docs_df))
    docs_per_topic = docs_df.groupby(['cluster'], as_index = False).agg({'text': ' '.join})
    return docs_df, docs_per_topic

In [ ]:
docs_df, docs_per_topic = clustering_documents(docs,hdbsan_cluster)
#docs_df, docs_per_topic = clustering_documents(docs,ac_cluster)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer



def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range).fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
c_tf_idf, count = c_tf_idf(docs_per_topic.text.values, m=len(docs))

def extract_top_n_words_per_topic(c_tf_idf, count, docs_per_topic, n=10):
    words = count.get_feature_names()
    labels = list(docs_per_topic.cluster)
    tf_idf_transposed = c_tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return words, top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['cluster'])
                     .text
                     .count()
                     .reset_index()
                     .rename({"cluster": "topic", "text": "size"}, axis='columns')
                     .sort_values("size", ascending=False))
    return topic_sizes

words,top_n_words = extract_top_n_words_per_topic(c_tf_idf, count, docs_per_topic, n=5)
topic_sizes = extract_topic_sizes(docs_df); 

# Maximal Marginal Relevance

In [ ]:
#top_n_words

In [ ]:
topics= pd.Series(hdbsan_cluster)
topics = topics.unique()
topics  = topics = sorted(list(topics))

In [ ]:
indices = c_tf_idf.argsort()[:, -30:]

In [ ]:
topics = {label: [(words[j], c_tf_idf[i][j])
                          for j in indices[i]][::-1]
                  for i, label in enumerate(labels)}

### Similarity Matrix

In [ ]:
import numpy as np
from typing import List
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.metrics.pairwise import cosine_similarity

import plotly.express as px
import plotly.graph_objects as go


def visualize_heatmap(topic_model,
                      topics: List[int] = None,
                      top_n_topics: int = None,
                      n_clusters: int = None,
                      width: int = 800,
                      height: int = 800) -> go.Figure:

    # Select topic embeddings
    if topic_model is not None:
        embeddings = np.array(topic_model)
    else:
        embeddings = c_tf_idf


    # Select topics based on top_n and topics args
    topics= pd.Series(topics)
    topics = topics.unique()
    topics = sorted(list(topics))
    """
     if topics is not None:
        topics= pd.Series(topics)
        topics = topics.unique()
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(topics[1:top_n_topics + 1])
    else:
        #topics= pd.Series(topics)
        #topics = topics.unique()
        #topics = sorted(list(topics))
   
   

        
    # Order heatmap by similar clusters of topics
    
    if n_clusters:
        if n_clusters >= len(set(topics)):
            raise ValueError("Make sure to set `n_clusters` lower than "
                             "the total number of unique topics.")

        embeddings = embeddings[[topic + 1 for topic in topics]]
        distance_matrix = cosine_similarity(embeddings)
        Z = linkage(distance_matrix, 'ward')
        clusters = fcluster(Z, t=n_clusters, criterion='maxclust')

        # Extract new order of topics
        mapping = {cluster: [] for cluster in clusters}
        for topic, cluster in zip(topics, clusters):
            mapping[cluster].append(topic)
        mapping = [cluster for cluster in mapping.values()]
        sorted_topics = [topic for cluster in mapping for topic in cluster]
    else:
        sorted_topics = topics
     """   
    sorted_topics = topics
    # Select embeddings
    indices = np.array([topics.index(topic) for topic in sorted_topics])
    #print(indices)
    embeddings = embeddings[indices]
    distance_matrix = cosine_similarity(embeddings)
    
    # Create nicer labels
    #print(index)
    #print(sorted_topics)

    new_labels = [[[str(topic), None]] + top_n_words[topic] for topic in sorted_topics]
    new_labels = ["_".join([label[0] for label in labels[:4]]) for labels in new_labels]
    new_labels = [label if len(label) < 30 else label[:27] + "..." for label in new_labels]
  
    fig = px.imshow(distance_matrix,
                    labels=dict(color="Similarity Score"),
                    x=new_labels,
                    y=new_labels,
                    color_continuous_scale='GnBu'
                    )

    fig.update_layout(
        title={
            'text': "<b>Similarity Matrix",
            'y': .95,
            'x': 0.55,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
    )
    fig.update_layout(showlegend=True)
    fig.update_layout(legend_title_text='Trend')

    return fig

In [ ]:
visualize_heatmap(topic_model=c_tf_idf,topics=hdbsan_cluster,width=1000, height=1000)

In [ ]:
visualize_heatmap(topic_model=bert_embeddings,topics=ac_cluster,width=1000, height=1000)

## BERTopic

In [ ]:
from sentence_transformers import SentenceTransformer, util

bert_model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens", device="cuda")

In [ ]:
from umap import UMAP
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine')

In [ ]:
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=24, 
                              metric='euclidean',                      
                               cluster_selection_method='eom',prediction_data=True).fit(umap_embeddings)

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model=bert_model,
                       #language = "multilingual",
                       top_n_words=5,
                       #n_gram_range=(1, 2),
                       #min_topic_size=15,   
                       #nr_topics = 10,
                       umap_model=umap_model,
                       #hdbscan_model=hdbscan_model,
                       low_memory=True,
                       calculate_probabilities=False, 
                       verbose=True)#.fit(processedTweets.text)

In [ ]:
topics, _ = topic_model.fit_transform(docs.text)

In [ ]:
# Save model
topic_model.save('./models/provaxxers-model')

# Load model
#my_model = BERTopic.load("my_model")	

In [ ]:
topic_model.visualize_topics()

## Topic Reduction

<ul>
    <li>Manual Topic Reduction</li>
    <li>Automatic Topic Reduction</li>
    <li><b>Topic Reduction after Training</b></li>
</ul> 


In [ ]:
newTopics, newProbs = topic_model.reduce_topics(docs.text, topics,  nr_topics= 19)

In [ ]:
topic_model.visualize_topics()

# Dynamic Topic Modeling

[Dou et al. 2012] define um evento como ”uma ocorrencia que causa mudancas no volume de dados de texto que discutem o topico associado em um momento especifico. Logo, eventos podem ser vistos como um resumo sucinto dos fluxos de informacoes nas mıdias sociais, revelando a evolucao de fenomenos sociais ao longo de um determinado periodo de tempo.

In [ ]:
timestamps = docs.created_at.to_list()
tweets = docs.text.to_list()

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                topics=newTopics, 
                                                #topics=topics,
                                                timestamps=timestamps, 
                                                global_tuning=False,
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)